In [3]:
import pandas as pd
import scipeds

scipeds.download_db()

from scipeds.data.completions import CompletionsQueryEngine
from scipeds.data.queries import (
    FieldTaxonomy,
    QueryFilters, 
)

engine = CompletionsQueryEngine()



Database already downloaded to /Users/kevinphan981/Library/Caches/.scipeds/scipeds_0_0_8.duckdb. To re-download and overwrite the existing file, re-run with `overwrite` set to `True`(from within Python) or add --overwrite to your CLI command.


In [87]:
from scipeds.constants import COMPLETIONS_TABLE
#  majornum,
completions = engine.get_df_from_query(f"""
    SELECT year, unitid, cipcode, awlevel, cip2020, race_ethnicity, gender, n_awards  
    FROM {COMPLETIONS_TABLE}
    WHERE YEAR BETWEEN 2010 AND 2019
    AND regexp_matches(CAST(CIPCODE AS VARCHAR), '^12\\.04.*$')
    ;
""")

completions_total = engine.get_df_from_query(f"""
    SELECT year, unitid, cipcode, awlevel, cip2020, race_ethnicity, gender, n_awards as n_awards_total 
    FROM {COMPLETIONS_TABLE}
    WHERE YEAR BETWEEN 2010 AND 2019
    ;
""")
print(completions.head())
print(completions.shape)

   year  unitid  cipcode                                            awlevel  \
0  2010  100760  12.0401  Award of at least 1 but less than 2 academic y...   
1  2010  101240  12.0401                 Award of less than 1 academic year   
2  2010  101240  12.0401                 Award of less than 1 academic year   
3  2010  101240  12.0401                 Award of less than 1 academic year   
4  2010  101240  12.0401                 Award of less than 1 academic year   

   cip2020             race_ethnicity gender  n_awards  
0  12.0401  Black or African American  women         3  
1  12.0401                    Unknown  women         1  
2  12.0401  Black or African American    men         1  
3  12.0401  Black or African American  women         2  
4  12.0401         Hispanic or Latino  women         1  
(146367, 8)


In [5]:
# to see the tables available

query = "SHOW TABLES"

engine.get_df_from_query(query) # this package doesn't offer that much, but it does at least offer the completions data

,name
0,cip_info
1,ipeds_completions_a
2,ipeds_directory_info


In [6]:
institution_fip = engine.get_institutions_table() # reads in all institutions and their features (we need state)
print(institution_fip.shape)

(9469, 102)


In [7]:
# creating index of cosmetology institutions
# cosmetology_schools = pd.merge(institution_fip, completions, how = "left", on = "unitid")
# wrong idea here, we need the unique schools from 'completions'

cosmetology_schools = completions['unitid'].unique()

print(cosmetology_schools.shape) # we cut down the amount by nearly a fourth


(2474,)


## Switching Gears to Pypeds

This is another library to help read in the data from IPEDS en masse. Whereas with the scipeds we could really only access the completions survey, we are able to access the other ones that we want.

I would like: 
1. Enrollment data
2. Retention rates

However, this is for cosmetology schools that are dedicated only to cosmetology, a university that has a cosmetology major would not count for this. I will do this by using the graduation / enrollment ratio. If it is above a threshold of .5, then it is likely a cosmetology school. I will vary the cutoff to see if anything changes.

In [69]:
import pypeds
from pypeds import ipeds

timeframe = range(2010, 2019, 1)
enrollment = ipeds.EFC(years = timeframe)
enrollment.extract()

enrollment_df = enrollment.load()
print(enrollment_df.head())
print(enrollment_df.shape)


     unitid  efcstate  line xefres01  efres01 xefres02 efres02  survey_year  \
0  100654.0       1.0   1.0        R    720.0        R     646       2010.0   
1  100654.0       2.0   2.0        R      2.0        R       1       2010.0   
2  100654.0       4.0   4.0        R      2.0        R       2       2010.0   
3  100654.0       5.0   5.0        R      1.0        R       1       2010.0   
4  100654.0       6.0   6.0        R      7.0        R       5       2010.0   

   fall_year  
0     2010.0  
1     2010.0  
2     2010.0  
3     2010.0  
4     2010.0  
(526136, 9)


In [ ]:
enrollment_cosme = enrollment_df[enrollment_df['unitid'].isin(cosmetology_schools)]
#enrollment_cosme = enrollment_cosme[['unitid', 'fall_year', 'xgrcohrt',  'grcohrt', 'xugenter',  'ugentern', 'xpgrcohr',  'pgrcohrt', 'xrrftct']]
print(enrollment_cosme.head())
print(enrollment_cosme.shape)

KeyError: "['xgrcohrt', 'grcohrt', 'xugenter', 'ugentern', 'xpgrcohr', 'pgrcohrt', 'xrrftct'] not in index"

In [71]:
print(enrollment_cosme[enrollment_cosme['unitid'] == 100760])

          unitid  efcstate   line xefres01  efres01 xefres02 efres02  \
185     100760.0       1.0    1.0        R    678.0        R     441   
186     100760.0       6.0    6.0        R      1.0        Z       0   
187     100760.0      12.0   12.0        R      2.0        R       2   
188     100760.0      13.0   13.0        R      3.0        R       3   
189     100760.0      47.0   47.0        R      2.0        R       2   
...          ...       ...    ...      ...      ...      ...     ...   
455897  100760.0      33.0   33.0        R      1.0        R     1.0   
455898  100760.0      42.0   42.0        R      1.0        R     1.0   
455899  100760.0      54.0   54.0        R      1.0        R     1.0   
455900  100760.0      58.0  999.0        R    372.0        R   305.0   
455901  100760.0      99.0   99.0        R    372.0        R   305.0   

        survey_year  fall_year  
185          2010.0     2010.0  
186          2010.0     2010.0  
187          2010.0     2010.0  
188

In [ ]:
# full_cosme_df = pd.merge(enrollment_cosme, completions, how = 'inner', on = 'unitid')
# full_cosme_df.shape
# TODO: Aggregate completions to year and institution, same with enrollment, regardless of demographic data.

completions_agg = completions.groupby(['unitid', 'year']).sum('n_awards').reset_index() #major num means nothing here.
completions_total_agg = completions_total.groupby(['unitid', 'year']).sum('n_awards_total').reset_index()

# not really necessary, but there.
enrollment_agg = enrollment_cosme.groupby(['unitid', 'fall_year']).sum('efres01').reset_index()
enrollment_agg['year'] = enrollment_agg['fall_year']

In [ ]:
full_cosme_agg = pd.merge(completions_agg, completions_total_agg, how = 'inner', on = ['unitid', 'year'])
full_cosme_agg.head()


full_cosme_agg['cosme_ratio'] = full_cosme_agg['n_awards']/full_cosme_agg['n_awards_total']

# conditional to filter

full_cosme_agg = full_cosme_agg[full_cosme_agg['cosme_ratio'] > .9] # no this isn't possible.

print(full_cosme_agg.shape)


full_cosme_agg_index = full_cosme_agg['unitid'].unique()
print(full_cosme_agg_index.shape)


(12147, 5)
(1705,)
